# Libs

In [1]:
import os
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

d:\DANG NHI\repos\master-llm-rag-vnlaw


In [7]:
import ollama
import asyncio
from ollama import AsyncClient

# Vector database và Embedding model

In [3]:
import chromadb
from chromadb import Settings, EmbeddingFunction, Embeddings

In [4]:
client = chromadb.PersistentClient(
    path="./chroma_db",
    settings=Settings(allow_reset=True,),)

client.list_collections()

['VNLaws']

In [5]:
collection = client.get_collection(name="VNLaws")
collection.count()

20921

In [6]:
base_model_id = "FacebookAI/xlm-roberta-base"
trained_model_path = "./models/trained_embedding_small_data/"
data_path = "data/"

MAX_LEN = 512
OVERLAP = 50

import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModel.from_pretrained(trained_model_path)
model.eval()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
model.to(device)

print("Model ready !!!")

Model ready !!!


In [8]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model, tokenizer, max_length=512):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length


    def embed_documents(self, texts):
        inputs = self.tokenizer(texts, 
                                padding='max_length', 
                                max_length=self.max_length, 
                                return_tensors="pt")
        with torch.no_grad():
            inputs = {key: value.to(device) for key, value in inputs.items()}
            embeddings = self.model(**inputs).pooler_output
    
        return embeddings.cpu().numpy()

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            inputs = {key: value.to(device) for key, value in inputs.items()}
            embeddings = self.model(**inputs).pooler_output
        return embeddings.cpu().numpy()

myembed = MyEmbeddingFunction(model, tokenizer)

In [9]:
# text = 'Tôi đi học'
# with torch.no_grad():
#     token = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=MAX_LEN)
#     output = model(**token)

output = myembed.embed_documents(["Tôi đi học"])
output1 = myembed.embed_query("Tôi đang viết trên bảng ở trường")

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(output.reshape(1, -1), output1.reshape(1, -1))
print(f"Cosine similarity:\n {similarity}")

Cosine similarity:
 [[0.82892543]]


# Create ollama client

In [10]:
ollama_api_url = "https://1ef6-34-168-188-244.ngrok-free.app/" # Thay đổi mỗi lần host

In [11]:
ollama_client = ollama.Client(
    host=ollama_api_url,
    headers = {'Header': 'application/json'}
)

In [15]:
system_prompt = "Xây dựng chương trình RAG với chatbot trung thực.\n" \
"Tôi có bộ dữ liệu gồm các Lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Lao Động, Nhà Đất và Tài Chính Ngân Hàng. " \
"Trong đó bao bồm các văn bản hành chính của Chính phủ.\n"

agent_judgment1 = 'Bạn là một agent dùng để đánh giá câu trả lời. ' \
'Với câu hỏi "{question}", Bạn có thể trả lời câu hỏi bằng cơ sở dữ liệu của tôi được hay không (trả lời 1 nếu có thể, trả lời 0 nếu không thế).' \
'Trả lời:' \

agent_search_plan = 'Bạn là một agent để dự đoán thông tin cần truy vấn.\n' \
'Theo bạn để trả lời câu hỏi "{question}", thì cần những thông tin gì và đến từ Lĩnh vực nào?\n' \
'Trả lời tối đa 3 dòng và theo cú pháp "<linh_vuc>: <thong_tin_can>", linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trùng Lĩnh vực nhưng khác nhau thông tin cần.\n' \
'Trả lời: '

agent_judgment2 = 'Bạn là một agent reranking' \

# question = "Quần đảo Trường Sa thuộc tỉnh nào?"
question = "Tôi có thể làm gì với hợp đồng lao động?"

In [16]:
# judment1: Trả lời được hay không
response = ollama_client.chat(
    model="gemma3:12b",
    messages=[{"role": "user", "content": system_prompt + agent_judgment1.format(question=question)}],
    stream=False,
)

print(response.message.content)

1



In [ ]:
def parse_judgment(response_text: str) -> int:
    for char in response_text.strip():
        if char in ['0', '1']:
            return int(char)
    return 0

if parse_judgment(response.message.content) == 1:
    print("Có thể trả lời được")

    response = ollama_client.chat(
        model="gemma3:12b",
        messages=[{"role": "user", "content": system_prompt + agent_search_plan.format(question=question)}],
        stream=False,
    )
    plans = response.message.content.split('\n')
    

else:
    print("Không thể trả lời được")

Lao Động: Các quyền và nghĩa vụ của người lao động và người sử dụng lao động theo hợp đồng lao động.
Nhà Đất: Các quy định pháp luật về quyền sở hữu và sử dụng tài sản liên quan đến hợp đồng lao động (nếu có).
Tài Chính Ngân Hàng: Các vấn đề về thanh toán, khấu trừ, và các nghĩa vụ tài chính khác phát sinh từ hợp đồng lao động.

Có thể trả lời được


In [ ]:
# PROMPT = agent_search_plan.format(question=question)



PROMPT = system_prompt + agent_search_plan.format(question=question)
print(PROMPT)

response = ollama_client.chat(
    model='gemma3:12b', 
    messages=[{'role': 'user','content': PROMPT,}],
    stream=False)

In [17]:
print(response.message.content)

<Nhà Đất>: Thông tin về phạm vi hành chính và địa lý Việt Nam để xác định tỉnh quản lý quần đảo Trường Sa.
<Tài Chính Ngân Hàng>: Các văn bản pháp luật liên quan đến quản lý và khai thác tài nguyên biển đảo, có thể đề cập đến việc xác định chủ quyền.
<Bảo Hiểm>: Các quy định về bảo hiểm cho hoạt động thăm dò, khai thác và bảo vệ chủ quyền trên quần đảo Trường Sa (gián tiếp liên quan đến xác định thuộc tỉnh nào).


Tôi có một sơ bộ dữ liệu gồm các lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Dân sự, Doanh Nghiệp và Giáo dục. Trong đó bao bồm các văn bản hành chính của Chính phủ.
Theo bạn để trả lời câu hỏi "Bảng giá dịch vụ khám chữa bệnh có bảo hiểm", thì cần những thông tin gì và đến từ Lĩnh vực nào?
Trả lời tối đa 3 dòng và theo mẫu <linh_vuc>: <thong_tin_can>, linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trung Lĩnh vực nhưng khác nhau thông tin cần.
Trả lời: 
